## 장식자 (Decorators)

In [1]:
def base(시작값):
    i = 0
    fn = lambda x, y: x + y + 시작값  # 익명함수
    return fn

In [5]:
def base(시작값):
    i = 0
    def fn(x, y):  # 익명함수 대신, 함수를 직정 정의하셔도 됩니다.
        return x + y + 시작값
    return fn

In [8]:
# 아래와 같이 만들 필요없이, base 함수를 활용하세요.
# def base_10(x, y):
#     return x + y + 10
# def base_20(x, y):
#     return x + y + 20
# def base_30(x, y):
#     return x + y + 30

base_10 = base(10)
base_20 = base(20)
base_30 = base(30)

In [4]:
def mysum1(x, y):
    return x + y + 10

mysum2 = lambda x, y: x + y + 10

print(mysum1(1, 2))
print(mysum2(1, 2))

13
13


### 정말 장식자 만들기

In [22]:
# base_10이라는 장식자 구현
def base_10(fn):
    def wrap(x, y):
        return fn(x, y) + 10
    return wrap

In [25]:
def mysum(x, y):
    return x + y

mysum = base_10(mysum)
mysum

<function __main__.base_10.<locals>.wrap(x, y)>

In [26]:
mysum(1, 2)

13

In [27]:
i = 10

i = i + 10

In [ ]:
def mysum(x, y):
    return x + y

mysum = base_10(mysum)

In [29]:
# 위 코드를 장식자 문법으로 아래와 같이 정리
@base_10
def mysum(x, y):
    return x + y

mysum(1, 2)

13

In [30]:
@base_10
@base_10
@base_10
def mysum(x, y):
    return x + y

mysum(1, 2)

33

아래 `mypower`함수에 `base_30` 장식자를 적용해보세요.

`mypower(2, 4)` 의 반환값은 `46`이 되어야 합니다.

In [34]:
# TODO: ...
def base_30(fn):
    def wrap(x, y):
        return fn(x, y) + 30
    return wrap

@base_30
def mypower(x, y):
    return x ** y

@base_30
def mymultiply(x, y):
    return x * y

print(mypower(2, 4))
print(mymultiply(2, 4))

46
38


### 장식자를 만들어주는 함수

In [43]:
def base(시작값):
    def wrap(fn):
        def inner(x, y):
            return fn(x, y) + 시작값
        return inner
    return wrap

In [44]:
@base(10)
@base(20)
@base(30)
def mysum(x, y):
    return x + y

mysum(1, 2)

63

장고에서의 `View`예시

```python
@gold_membership_required
def post_list(request):
    qs = Post.objects.all()
    return render(request, 'blog/post_list.html', {
        'post_list': qs,
    })
```

### 예시: Memoize

In [47]:
import time

def mysum2(x, y):
    time.sleep(1)
    return x + y

print(mysum2(1, 2))
print(mysum2(1, 2))
print(mysum2(1, 2))

print(mysum2(1, 3))

3
3
3
4


계산결과를 `기억`하도록, 개선해봅시다.

In [54]:
import time

cached = {}

def mysum2(x, y):
    key = (x, y)
    if key not in cached:
        time.sleep(1)
        cached[key] = x + y
    return cached[key]

cached2 = {}

def mymultiply(x, y):
    key = (x, y)
    if key not in cached2:
        time.sleep(1)
        cached2[key] = x * y
    return cached2[key]

print(mysum2(1, 2))
print(mysum2(1, 2))
print(mysum2(1, 2))

print(mysum2(1, 3))

print(mymultiply(1, 2))
print(mymultiply(1, 2))
print(mymultiply(1, 2))

3
3
3
4
2
2
2


장식자 문법으로 깔끔하게 만들어봅시다.

In [57]:
import time

def memoize(fn):
    cached = {}
    def wrap(x, y):
        key = (x, y)
        if key not in cached:
            cached[key] = fn(x, y)
        return cached[key]
    return wrap

@memoize
def mysum2(x, y):
    time.sleep(1)
    return x + y
# mysum2 = memoize(mysum2)

@memoize
def mymultiply(x, y):
    time.sleep(1)
    return x * y
# mymultiply = memoize(mymultiply)

print(mysum2(1, 2))
print(mysum2(1, 2))
print(mysum2(1, 2))

print(mysum2(1, 3))

print(mymultiply(1, 2))
print(mymultiply(1, 2))
print(mymultiply(1, 2))

3
3
3
4
2
2
2


## 문자열 인코딩

In [59]:
이름 = "이진석"  # 유니코드 문자열 (str 타입)

print(type(이름))
print(len(이름))

<class 'str'>
3


In [67]:
utf8_bytes = 이름.encode("utf8")  # bytes 타입
print(type(utf8_bytes), len(utf8_bytes))
print(utf8_bytes)

utf8_bytes.decode('cp949')

<class 'bytes'> 9
b'\xec\x9d\xb4\xec\xa7\x84\xec\x84\x9d'


UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 0: illegal multibyte sequence

In [65]:
cp949_bytes = 이름.encode("cp949")  # bytes 타입
print(type(cp949_bytes), len(cp949_bytes))
cp949_bytes

<class 'bytes'> 6


b'\xc0\xcc\xc1\xf8\xbc\xae'

In [71]:
s = "abc 가나다라마바사아자차타카파하"
s[1:-1]

'bc 가나다라마바사아자차타카파'

In [78]:
print('0x%X' % ord("ㄱ"))
print('0x%X' % ord("ㅏ"))
print('0x%X' % ord("가"))

0x3131
0x3134
0x314F
0xAC00


In [79]:
print("가".encode("utf8"))
print("가".encode("utf16"))
print("가".encode("utf32"))
print("가".encode("cp949"))

b'\xea\xb0\x80'
b'\xff\xfe\x00\xac'
b'\xff\xfe\x00\x00\x00\xac\x00\x00'
b'\xb0\xa1'


## 파일 I/O

In [85]:
# file mode : write + binary
f = open("파일01.txt", "wb")
f.write("한글\n".encode("utf8"))
f.write("파이썬\n".encode("utf8"))
f.write("장고\n".encode("utf8"))
f.write("크롤링".encode("utf8"))
f.close()

In [88]:
# file mode : write + text
f = open("파일01.txt", "wt", encoding='utf8')
f.write("한글\n")
f.write("파이썬\n")
f.write("장고\n")
f.write("크롤링")
f.close()

In [93]:
f = open("파일01.txt", "rb")
readed = f.read()
print('{} bytes readed.'.format(len(readed)))
print(readed)   # 한 방에 다 읽습니다.
f.close()

print(readed.decode('utf8'))

33 bytes readed.
b'\xed\x95\x9c\xea\xb8\x80\n\xed\x8c\x8c\xec\x9d\xb4\xec\x8d\xac\n\xec\x9e\xa5\xea\xb3\xa0\n\xed\x81\xac\xeb\xa1\xa4\xeb\xa7\x81'
한글
파이썬
장고
크롤링


In [92]:
f = open("파일01.txt", "rt", encoding='utf8')
readed = f.read()
print('{} 글자 readed.'.format(len(readed)))
print(readed)   # 한 방에 다 읽습니다.
f.close()

13 글자 readed.
한글
파이썬
장고
크롤링
